#  프로젝트 (1) moviepy로 비디오 처리하기

In [1]:
from moviepy.editor import VideoClip, VideoFileClip
from moviepy.editor import ipython_display
import cv2
import numpy as np
import os

In [2]:
# 읽기
video_path = os.getenv('HOME')+'/aiffel/video_sticker_app/images/video2.mp4'
clip = VideoFileClip(video_path) # moviepy로 읽어오는 과정 
clip = clip.resize(width=640) # 크기 설정 
clip.ipython_display(fps=30, loop=True, autoplay=True, rd_kwargs=dict(logger=None))
# video의 여러가지 설정들 

# 쓰기
result_video_path = os.getenv('HOME')+'/aiffel/video_sticker_app/images/mvpyresult.mp4'
clip.write_videofile(result_video_path)

t:   0%|          | 0/404 [00:00<?, ?it/s, now=None]Moviepy - Building video /home/aiffel-dj2/aiffel/video_sticker_app/images/mvpyresult.mp4.
MoviePy - Writing audio in mvpyresultTEMP_MPY_wvf_snd.mp3
MoviePy - Done.
Moviepy - Writing video /home/aiffel-dj2/aiffel/video_sticker_app/images/mvpyresult.mp4

Moviepy - Done !
Moviepy - video ready /home/aiffel-dj2/aiffel/video_sticker_app/images/mvpyresult.mp4


## moviepy 

In [4]:
# CASE 1 : moviepy 사용
start = cv2.getTickCount()
clip = VideoFileClip(video_path)
clip = clip.resize(width=640)

vlen = int(clip.duration*clip.fps)
video_container = np.zeros((vlen, clip.size[1], clip.size[0], 3), dtype=np.uint8)

for i in range(vlen):
    img = clip.get_frame(i/clip.fps)
    video_container[i] = (img * 0.5).astype(np.uint8) # 밝기?  

# 새 clip 만들기
dur = vlen / clip.fps
outclip = VideoClip(lambda t: video_container[int(round(t*clip.fps))], duration=dur)
# 쓰기
mvpy_video_path = os.getenv('HOME')+'/aiffel/video_sticker_app/images/mvpyresult.mp4'
outclip.write_videofile(mvpy_video_path, fps=30)

time = (cv2.getTickCount() - start) / cv2.getTickFrequency()
print (f'[INFO] moviepy time : {time:.2f}ms')

t:  12%|█▏        | 48/403 [00:00<00:00, 438.82it/s, now=None]Moviepy - Building video /home/aiffel-dj2/aiffel/video_sticker_app/images/mvpyresult.mp4.
Moviepy - Writing video /home/aiffel-dj2/aiffel/video_sticker_app/images/mvpyresult.mp4

Moviepy - Done !
Moviepy - video ready /home/aiffel-dj2/aiffel/video_sticker_app/images/mvpyresult.mp4
[INFO] moviepy time : 3.05ms


## OpenCV 

In [5]:
# CASE 2 : OpenCV 사용
start = cv2.getTickCount()
vc = cv2.VideoCapture(video_path)

cv_video_path = os.getenv('HOME')+'/aiffel/video_sticker_app/images/cvresult.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
vw = cv2.VideoWriter(cv_video_path, fourcc, 30, (640,360))

vlen = int(vc.get(cv2.CAP_PROP_FRAME_COUNT))

for i in range(vlen):
    ret, img = vc.read()
    if ret == False: break
    
    img_result = cv2.resize(img, (640, 360)) * 0.5
    vw.write(img_result.astype(np.uint8))
    
time = (cv2.getTickCount() - start) / cv2.getTickFrequency()
print (f'[INFO] cv time : {time:.2f}ms')

[INFO] cv time : 1.63ms


## moviepy 를 이용할 때의 장단점을 분석해 봅시다.

- 단점은 두배정도 느리다 때문에 실시간 비디오나 대용량 비디오 처리에는 적합하지 않다. 
- 장점은 numpy를 사용할 수 있는 장점이 있고 그래서 다양한 효과를 추가 할 수 있습니다. 


# 어디까지 만들고 싶은지 정의하기 

## 스티커앱을 실행하고 카메라를 고정하고 서서히 멀어져봅니다. 혹은 아주 가까이 다가가 봅니다. 얼굴을 찾지 못하는 거리를 기록해주세요

- 거리는 팔 길이 이상 한걸음 반정도 갔음에도 인식이 잘 되는 편이였다. 
- 가까이 갔을땐 턱이 짤리는 경우 인식을 하지 못하였다. 

## 다시 자리로 돌아온 후 고개를 상하좌우로 움직여주세요. yaw, pitch, roll 각도의 개념을 직접 실험해 보고 각각 몇 도까지 정상적으로 스티커앱이 동작하는지 기록해주세요.

- yaw : 짤리지 않는 이상 잘 되었다. . 
- picth : X축 기준 회전(좌우)는 왼쪽눈이 보이면 머리위가 아니라 이마 중심으로 공중에 떠있는 왕관을 볼수 있었다. 45도 이상 거의 90 전까지는 그래도 인식은 하는 편이라고 봣다. 
- roll : 제일 민감 턱과 눈썹 하나라도 보이지 않으면 안됬다. 

## 만들고 싶은 스티커앱의 스펙(허용 거리, 허용 인원 수, 허용 각도, 안정성)을 정해주세요.

허용거리

- 허용 거리 손과 셀카봉길이를 합친 길이 만큼 떨어져도 인식이 잘되는 정도 
- 얼굴 부위가 짤리는 수준으로 가까우면 인식이 되지 않는 정도 

허용 인원수
- 4~5인 까지 얼굴이 한 화면에 담기 어렵고 겹칠 가능성이 있기 때문에 5인 정도 인식 하는 수준

허용 각도 
- 핸드폰을 직접 보기 힘든 각도까지 즉 90도 전 80도 정도 까지 인식 하는 수준 

안정성 
- 위 조건들을 만족하면서 200장당 1번의 에러 정도 

# 스티커 Out Bound 예외처리 하기 

## 지금까지 만든 스티커앱을 이용해서 예외 상황을 찾아주세요. 특히 서서히 영상에서 좌우 경계 밖으로 나가며 코드의 행동을 확인해 보세요.

```
cv2.error: OpenCV(4.5.1) /tmp/pip-req-build-hj027r8z/opencv/modules/core/src/arithm.cpp:666: error: (-209:Sizes of input arguments do not match) The operation is neither 'array op array' (where arrays have the same size and the same number of channels), nor 'array op scalar', nor 'scalar op array' in function 'arithm_op'

```
array 범위를 벗어났다는 오류 코드가 나오면서 작동이 정지 된다. 

## 문제가 어디에서 발생하는지 코드에서 확인합니다.
- newaddsticker.py의 img2sticker 메소드에서 아래 부분에 
``` 
   if refined_y < 0:
        img_sticker = img_sticker[-refined_y:]
        refined_y = 0
```

이 부분에 추가로 조건문을 더 달아 줘야 한다. 

## Out bound 오류(경계 밖으로 대상이 나가서 생기는 오류)를 해결해 주세요.

- 위 조건문을 아래와 같이 수정해 준다. 

```
    if refined_x < 0:
        img_sticker = img_sticker[:, -refined_x:]
        refined_x = 0
    elif refined_x + img_sticker.shape[1] >= img_orig.shape[1]:
        img_sticker = img_sticker[:, :-(img_sticker.shape[1]+refined_x-img_orig.shape[1])]
```


## 다른 예외는 어떤 것들이 있는지 정의해 주세요. 어떤 것이 문제가 되는지 스스로 정해봅시다.

아래 하단으로 얼굴을 찍을 시 머리 밖 공중에 왕관이 씌워져 있다. 

# 칼만 필터 적용하기 
## 카메라 앞에서 가만히 있을 때 스티커의 움직임을 관찰해 주세요. 어떤 문제가 발생하나요?

- 가만히 있어도 스티커의 크리가 일정하게 유지되지 않고, 떨리는 것처럼 보이는 현상이 발생합니다.

## 이론 강의에서 배운 칼만 필터를 적용해서 스티커 움직임을 안정화 시켜주세요. 
- 칼만필터를 적용 후에는 왕관이 생기지 않았다 무슨 문제 인지 파악하기 어렵다. 
- [webcam_sticker.py](https://github.com/moon-jaeyun/aimoons/blob/master/webcam_sticker.py)